#### **PyTorch**

In [ ]:
import torch
import torch.nn.functional as F
print(f"PyTorch Version: {torch.__version__}")

import torch
print(f"Cude is available: {torch.cuda.is_available()}")
print(f"Device name: {torch.cuda.get_device_name(0)}")

#### **Import Other Libraries**

In [ ]:
from datasets import load_dataset 
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import numpy as np
import evaluate
import transformers
from transformers import TrainingArguments
import torch 
import matplotlib.pyplot as plt 
from transformers import DataCollatorWithPadding
import os 
from pathlib import Path
import random 
from datasets import Dataset, DatasetDict
import warnings
from functools import partial
from datasets import concatenate_datasets
from functools import partial 
from tqdm import tqdm 
import textwrap
from IPython.display import display
from IPython.display import Markdown
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from peft import LoraConfig, get_peft_model 
from transformers import BitsAndBytesConfig
import os 
import re 
os.environ["WANDB_DISABLED"] = "true"
warnings.filterwarnings('ignore', message='Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.')
from llmft.train import DecoderTrainer, EarlyStopping
from llmft.metrics import compute_recall
from llmft.losses import FocalLoss
from llmft.utils import predict, log_predictions
from llmft.generate import generate_dataset
from llmft.paper import generate_recall_table

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

#### **Parameters**

In [ ]:
# This cell is tagged with `parameters`
"meta-llama/Meta-Llama-3-8B-Instruct" #"google/gemma-1.1-7b-it" #microsoft/phi-2" #"microsoft/phi-2" #"#"meta-llama/Llama-2-7b-chat-hf" # "distilbert-base-uncased" 
model_name = "microsoft/Phi-3-mini-4k-instruct"
column = 'text'
epochs = 1
seed = 0
verbose = True 
test_size = 0.5
p = 0.0

#### **Qlora**

In [ ]:
from peft import LoraConfig, get_peft_model 
from transformers import BitsAndBytesConfig

# ----- QUANTIZATION -------# 
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# ----- LORA -------# 

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

#### **Instantiate Model**

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                             device_map="auto", 
                                             quantization_config=bnb_config, 
                                             trust_remote_code=True)# So we can do gradient checkpointing
model.config.use_cache = False
model.config.pretraining_tp = 1
model.config.gradient_checkpointing = True
model.enable_input_require_grads()
print(model.generation_config)

model = get_peft_model(model, lora_config)
print(model.print_trainable_parameters())

#### **Tokenizer**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
def tokenizer_function(example):
  return tokenizer.apply_chat_template(example["messages"], 
                                                          tokenize=True, 
                                                          add_generation_prompt=False, 
                                                          return_dict=True)

#### **Hyperparameters**

In [ ]:
lr = 1e-5
warmup_ratio = 0
batch_size = 4
epochs = 30
patience = float('inf') 
gamma = 0.0
training_status = 'standard' if lr==1e-4 else 'preferred'

#### **Data set**

In [ ]:
random_dataset =  [[{'content': 'You are a housing court clerk', 'role': 'system'},
                    {'content': "Task: The following is a description of an eviction case. Predict whether the tenant has legal represenation (yes or no, and then explain your reasoning.)\n\n    Description: Yes, the Right to Counsel is in effect in the tenant's zip code. The tenant violated the lease.\n\n    Prediction:",'role': 'user'},
                    {'content': 'No', 'role': 'assistant'}],
                  [{'content': 'You are a housing court clerk', 'role': 'system'},
                    {'content': "Task: The following is a description of an eviction case. Predict whether the tenant has legal represenation (yes or no, and then explain your reasoning.)\n\n    Description: Yes, the Right to Counsel is in effect in the tenant's zip code. The tenant lost their job because of health related accident.\n\n    Prediction:",'role': 'user'},
                    {'content': 'No', 'role': 'assistant'}],
                  [{'content': 'You are a smart AI', 'role': 'system'},
                    {'content': "In three words, in Economics, the acronym 'llm' stands for what? ", 'role': 'user'},
                    {'content': 'Joe Biden', 'role': 'assistant'}],
                  [{'content': 'You are a smart AI', 'role': 'assistant'},
                    {'content': "In one word, how good at Basketball is Lebron James?", 'role': 'user'},
                    {'content': 'Excellent', 'role': 'assistant'}],
                    ]
random_dataset =  Dataset.from_dict({'messages': random_dataset})

In [ ]:
# dataset = load_dataset("ppower1/chat_instrument")['train']
# labels = np.array([1 if i[2]['content'] == 'Yes' else 0 for i in dataset['messages']])
dataset = generate_dataset(total_entries=1000, flip_rate=0.)

def get_prompt(desc):
    return f"""Task: The following is a description of an eviction case. Predict whether the tenant has legal represenation (yes or no, and then explain your reasoning.)

    Description: {desc}

    Prediction:"""

messages = []
for i, j in zip(dataset['text'], dataset['label']):
   message = [{"role": "system", "content": "You are a housing court clerk"}, 
   {"role": "user", "content": get_prompt(i)}, 
   {"role": "assistant", "content": 'Yes' if j == 1 else 'No'}]
   messages.append(message)

dataset = Dataset.from_dict({'messages': messages})



dataset = dataset.select(range(1000))
# Reshuffle and split the combined dataset with a fixed seed
dataset = dataset.train_test_split(test_size=test_size, seed=seed)  # adjust test_size as needed
tokenized_dataset = dataset.map(tokenizer_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['messages'])

#### **Data Loaders**

In [ ]:
train_loader = DataLoader(tokenized_dataset['train'], batch_size=batch_size, collate_fn=DataCollatorWithPadding(tokenizer), shuffle=True)
test_loader = DataLoader(tokenized_dataset['test'], batch_size=batch_size, collate_fn=DataCollatorWithPadding(tokenizer))

#### **Optimizer**

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler =transformers.optimization.get_linear_schedule_with_warmup(optimizer,int(warmup_ratio*len(train_loader)*epochs), len(train_loader)*epochs)

#### **Class Weights**

In [ ]:
class_weights = torch.tensor([1., 1.], device=device) #torch.tensor([1-n_positive/n_samples, n_positive/n_samples], device=device)
criterion = FocalLoss(alpha=class_weights, gamma=gamma, mode='output', reduction='none')
criterion = torch.nn.CrossEntropyLoss(reduction='mean') 

#### **Trainer**

In [ ]:
yuri = DecoderTrainer(model,
                    optimizer, 
                    scheduler,
                    criterion, 
                    device, 
                    verbose=False, 
                    threshold=10)

print(yuri.verbose)

In [ ]:
evaluation_losses = [yuri.evaluate(test_loader)]
training_losses = [yuri.evaluate(train_loader)]
pbar =  tqdm(range(epochs), desc=f'Epoch: 0, Train Loss: {training_losses[0]:.3f}, Val Loss: {evaluation_losses[0]:.3f}')

for epoch in pbar:
    log_predictions(model, tokenizer, device, random_dataset, './log_output.txt')
    train_loss = yuri.train(train_loader)
    training_losses.append(train_loss)
    val_loss = yuri.evaluate(test_loader)
    evaluation_losses.append(val_loss)
    pbar.set_description(f'Epoch: {epoch + 1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

In [ ]:
plt.plot(training_losses)
plt.plot(evaluation_losses)
plt.show()

In [ ]:


for i in [X1, X2]:
    x = i[:2]
    target = i[2]['content']
    inputs = tokenizer.apply_chat_template(x, add_generation_prompt=True, return_tensors='pt').to(device)
    outputs = yuri.model.generate(inputs, max_new_tokens=100)
    text = tokenizer.batch_decode(outputs)[0].split('<|assistant|> \n')[1]
    print(text)

In [ ]:
# raw_targets = []
# score = []

# for i in range(100):
#     x = dataset['test']['messages'][i]
#     target = x[2]['content']
#     inputs = tokenizer.apply_chat_template(x, add_generation_prompt=True, return_tensors='pt').to(device)
#     outputs = yuri.model.generate(inputs, max_new_tokens=100)
#     text = tokenizer.batch_decode(outputs)[0].split('<|assistant|> \n')[1]
#     raw_targets.append(1 if target == 'Yes' else 0)
#     score.append(1 if text.startswith(target) else 0)
#     print(f"Target: {target}    |   {text.startswith(target)}")
